In [71]:
from keras.models import load_model
import tensorflow as tf
from fastapi import FastAPI, Body
from fastapi.middleware.cors import CORSMiddleware

from pydantic import BaseModel

from json import loads

from keras.models import load_model

import tensorflow as tf
import numpy as np
import spacy

from Objects.Functions.get_word_pairs import get_wp_in_line_hard
import pandas as pd
from string import punctuation
import re
import string
from scipy.spatial.distance import cosine

In [72]:
tf.config.run_functions_eagerly(False)
nlp_type_of_event_extraction = spacy.load('../../PipeLines/WordsExtraction/words_extract/')
nlp_classic = spacy.load('ru_core_news_sm')
kcm_extraction_model = tf.keras.models.load_model(
    '../../PipeLines/Coder/coder_from_spacy_to_kcm_onh5.h5')
# kcm_extraction_model.evaluate
is_description_model = load_model('../../PipeLines/Classifications/checker_is_discriptor_spacy_vectorize.h5')

In [73]:
def get_vectorized_wp_and_wp(text, vectorizer):
        list_of_wp = list()
        list_of_vectors = list()
        for sentence in text.split('.'):
            for wp in get_wp_in_line_hard(sentence):
                list_of_vectors.append(vectorizer(wp))
                list_of_wp.append(wp)
        return np.array(list_of_wp), np.array(list_of_vectors)

In [74]:
def remove_punctuation(descriptors_name):
    desc = list()
    for i in descriptors_name:
        desc.append(i.translate(str.maketrans('','', string.punctuation)))
    return desc

In [75]:
def get_distances_between_wp(descriptor_vector, metric_function):
    final_summ = list() 
    for i in range(len(descriptor_vector)):
        for j in range(i + 1, len(descriptor_vector)):
            cos = metric_function(descriptor_vector[i], descriptor_vector[j])
            final_summ[i].append(cos)
            final_summ[j].append(cos)
    return final_summ

In [76]:
def sendStudentJson(description):
    wp_of_event, wp_of_event_vectorized = get_vectorized_wp_and_wp(description, lambda text: nlp_classic(text).vector)
    is_wp_descriptor = is_description_model.predict(wp_of_event_vectorized) > 0.992
    descriptors_vectorized = wp_of_event_vectorized[is_wp_descriptor.T[0]]
    predict = kcm_extraction_model.predict(descriptors_vectorized)
    #descriptors_name
    descriptors_name = wp_of_event[is_wp_descriptor.T[0]]
    
    description_name_without_punct = remove_punctuation(descriptors_name)
    
    vectorized_description_name_without_punctuation = vectorized_description_name_without_punct(remove_punctuation)
    n = 0
    final_summ = [list() for i in vectorized_description_name_without_punctuation]

    vectorize_end = stdotkl(final_summ)
    res = np.where(vectorize_end == np.min(vectorize_end))[0]
    description_name_without_punct[int(res)]
    
    return description_name_without_punct[int(res)], kcm_extraction_model.predict(np.array([nlp_classic(description_name_without_punct[int(res)]).vector]))

In [77]:
abc = 'В форме представлений типов мышления, обсуждение их особенностей, применительности к реальным задачам, сравнительные характеристики.'

In [78]:
wp,  vwp = get_vectorized_wp_and_wp(abc, lambda text: nlp_classic(text).vector)

In [79]:
is_wp_descriptor = is_description_model.predict(vwp) > 0.992

3/3 [==============================] - 0s 5ms/step


In [80]:
dv = vwp[is_wp_descriptor.T[0]]
d = wp[is_wp_descriptor.T[0]]

In [81]:
predict = kcm_extraction_model.predict(dv)

1/1 [==============================] - 0s 149ms/step


In [82]:
predict

array([[-2.7230851e-02,  1.1530252e+00,  9.3390599e-02],
       [ 2.3817621e-02,  1.1737771e+00, -2.2050440e-03],
       [-1.7028779e-02,  1.0819974e+00,  5.9703752e-02],
       [ 9.7290501e-03,  9.8988956e-01,  2.4214387e-04]], dtype=float32)

In [83]:
from Objects.Functions.get_word_pairs import get_wp_in_line_hard, get_vectorized_wp_and_wp
from Objects.Functions.get_types_of_event import get_types_of_event_spacy
from Objects.Functions.get_wp_distances import get_distances_between_wp

In [84]:
distances = np.argsort(list(map(lambda a: np.array(a).std(), get_distances_between_wp(descriptor_vector=dv, metric_function=cosine))))
predict = kcm_extraction_model.predict(dv[distances[0:2]])

1/1 [==============================] - 0s 46ms/step


In [85]:
distances

array([1, 2, 3, 0], dtype=int64)

In [86]:
dv[distances[0:2]]

array([[-0.59135526, -0.18395996,  0.17253928, -0.04396268,  0.9353698 ,
         0.18774764,  1.2401947 ,  0.4095701 ,  0.21220809, -0.9230838 ,
        -0.15536904, -1.2114451 , -1.3029203 ,  1.4868159 , -0.28237996,
        -0.1745209 , -0.43754986, -0.73927516, -1.568915  ,  1.3296856 ,
         0.5218561 , -0.6542479 ,  0.6036532 ,  1.0517882 , -0.8474532 ,
        -0.20859516,  0.22622108, -0.43271026,  0.29257143, -0.5356993 ,
         0.34807053, -0.7714756 , -0.8102071 ,  0.6690045 , -0.3225636 ,
         0.3268718 , -2.135021  , -0.49551725,  0.2524806 , -0.8629706 ,
        -0.41905904,  0.43737972,  0.2783629 , -0.8816537 , -0.44250044,
        -1.4522643 , -0.2226783 , -1.0433569 , -0.5548043 ,  0.9303481 ,
        -0.2740887 ,  0.09103168, -0.06262046, -0.5992047 ,  1.4300972 ,
         0.28260157, -0.5352357 ,  0.55341023, -0.694973  ,  1.1642292 ,
        -0.20101662, -0.21450822,  0.11587569,  0.22602427,  1.0045058 ,
         0.16885489, -0.24806218,  0.30139938,  0.1

In [87]:
dv[distances[0:2]].shape, type(dv[distances[0:2]]), type(dv[distances[0:2]][0]), dv[distances[0:2]][0].shape

((2, 96), numpy.ndarray, numpy.ndarray, (96,))

In [88]:
predict

array([[ 0.02381767,  1.1737773 , -0.002205  ],
       [-0.0170289 ,  1.0819975 ,  0.05970396]], dtype=float32)

In [89]:
tf.config.run_functions_eagerly(True)

In [94]:
for sentence in abc.split('.'):
    print(1, sentence, len(sentence))
    get_wp_in_line_hard(sentence)

1 В форме представлений типов мышления, обсуждение их особенностей, применительности к реальным задачам, сравнительные характеристики 131
1  0


In [90]:
skills = dict()
for sentence in abc.split('.'):
    wp_of_event, wp_of_event_vectorized = get_vectorized_wp_and_wp(word_pairs=get_wp_in_line_hard(sentence), 
                                                                    vectorizer=lambda text: nlp_classic(text).vector)
    if len(wp_of_event):
        is_wp_descriptor = is_description_model.predict(wp_of_event_vectorized)
        print(is_wp_descriptor.T[0]>0.5)
        descriptors_vectorized = wp_of_event_vectorized[is_wp_descriptor.T[0]>0.5]
        descriptors_name = wp_of_event[is_wp_descriptor.T[0]>0.5]
        # print(is_wp_descriptor)
        if len(descriptors_name) != 0:
            distances = np.argsort(list(map(lambda a: np.array(a).std(), 
                                            get_distances_between_wp(descriptor_vector=descriptors_vectorized, 
                                                                    metric_function=cosine))))
            # print(distances)
            print(descriptors_vectorized[distances.T[0]], 
                type(descriptors_vectorized[distances[0:2]]), 
                descriptors_vectorized[distances[0:2]][0].shape, 
                descriptors_vectorized[distances.T[0:2]].shape)
            # print(descriptors_vectorized[distances.T[0]])
            predict = kcm_extraction_model.predict(descriptors_vectorized[distances[0:2]], )
            for name, kcm in zip(descriptors_name, predict):
                skills[name] = {'know': float((kcm[1]+1)/2), 'can': float((kcm[2]+1)/2), 'master': float((kcm[0]+1)/2)}
            print('\n')
        

3/3 [==============================] - 0s 13ms/step
[False False False False False False False False False False False False
  True  True  True  True  True False False False False False False  True
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False  True False False
 False False False False False False]
[ 0.56233007  0.39186108  0.16466665 -0.9715398   0.8060682  -0.14496042
  0.5426063  -0.81216353 -0.11705708 -0.04717104 -0.30298313  0.08511698
  0.17209752  0.5559431   0.81157017  0.292975    0.29184234  0.87144583
  0.04807131  0.3588399  -0.16138585 -0.7049678   0.47095597 -0.24363661
  0.49398562 -1.0977315  -0.20614164  0.826577   -1.355505    0.42217708
  0.05793822 -0.13716026  0.40736198 -0.11919902  0.26453623 -0.5062477
  0.2281958  -1.3768774  -0.0711289

In [91]:
skills

{'форме представлений': {'know': 0.5567205362021923,
  'can': 0.9712267518043518,
  'master': 0.4931637980043888},
 'форме представлений типов': {'know': 1.0765125751495361,
  'can': 0.5466952472925186,
  'master': 0.48638464137911797}}